<a href="https://www.kaggle.com/shreybandlish/notebook2d7e3d1812?scriptVersionId=88776185" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


**Importing Libraries**

In [30]:
import cuml
import cudf as pd
import cupy as cp
from cuml import PCA
from cuml.decomposition import PCA
from cuml.linear_model import LinearRegression
from cuml.model_selection import train_test_split
import seaborn as sns

**Loading Data**

In [31]:
train_df=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test_df=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

**Data Preprocessing**

In [32]:
numerical_features = train_df.dtypes[train_df.dtypes != "object"].index
print("No. of numerical features  ", len(numerical_features))

No. of numerical features   38


In [33]:


categorical_features = train_df.dtypes[train_df.dtypes == "object"].index
print("No. of categorical features  ", len(categorical_features))

No. of categorical features   43


In [34]:
for a in list(categorical_features) :
    print(train_df[a].value_counts())
    print('////////////////////////////////////////////////////////2')

RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int32
////////////////////////////////////////////////////////2
Pave    1454
Grvl       6
Name: Street, dtype: int32
////////////////////////////////////////////////////////2
Grvl    50
Pave    41
Name: Alley, dtype: int32
////////////////////////////////////////////////////////2
Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int32
////////////////////////////////////////////////////////2
Lvl    1311
Bnk      63
HLS      50
Low      36
Name: LandContour, dtype: int32
////////////////////////////////////////////////////////2
AllPub    1459
NoSeWa       1
Name: Utilities, dtype: int32
////////////////////////////////////////////////////////2
Inside     1052
Corner      263
CulDSac      94
FR2          47
FR3           4
Name: LotConfig, dtype: int32
////////////////////////////////////////////////////////2
Gtl    1382
Mod      65
Sev      13
Name: LandSlope, dtype: i

In [35]:
feature_list_mode = ['BsmtCond','BsmtQual','FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','MasVnrType','MasVnrArea','BsmtExposure','BsmtFinType2','Utilities','Exterior1st','Exterior2nd','BsmtFinType1','SaleType','BsmtFullBath','BsmtHalfBath','KitchenQual','Functional',]
feature_list_mean = ['GarageCars','GarageArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','LotFrontage']


In [36]:
train_df.drop(['Alley','PoolQC','Fence','MiscFeature','GarageYrBlt','Id'],axis=1,inplace=True)
test_df.drop(['Alley','PoolQC','Fence','MiscFeature','GarageYrBlt','Id'],axis=1,inplace=True)

In [37]:
for a in feature_list_mode:
    train_df[a] = train_df[a].fillna(train_df[a].mode()[0])
    


In [38]:
for a in feature_list_mean:
    train_df[a] = train_df[a].fillna(train_df[a].mean())

In [39]:
for a in feature_list_mode:
    test_df[a] = test_df[a].fillna(test_df[a].mode()[0])
    


In [40]:
for a in feature_list_mean:
    test_df[a] = test_df[a].fillna(test_df[a].mean())

In [41]:
train_df.dropna(inplace=True)

In [42]:
train_df.isnull().sum()

MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
Street           0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 75, dtype: uint64

In [43]:
final_data = pd.concat([train_df, test_df],axis=0)

In [44]:
final_data1 = pd.get_dummies(final_data)

In [45]:
for column in final_data1.columns:
    if final_data1[column].dtype == 'O':
        final_data1.drop([column], axis=1, inplace=True)

In [46]:
Train=final_data1.iloc[:1459,:]
Test=final_data1.iloc[1459:,:]

In [47]:
X=Train.drop(['SalePrice'],axis=1)
Y=Train['SalePrice']
Test=Test.drop(['SalePrice'],axis=1)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.29)

In [49]:
linreg = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd-jacobi')
split=linreg.fit(X_train, y_train)

In [50]:
print("R-Squared Value for Training Set:",(linreg.score(X_train.astype('float32'), y_train.astype('float32'))))

R-Squared Value for Training Set: 0.9307368397712708


In [51]:
y_pred_split=linreg.predict(Test)
y_pred_split

0       110306.315368
1       160644.917804
2       191337.705044
3       196806.245010
4       197996.488006
            ...      
1454     77062.646434
1455     67807.882063
1456    181977.519468
1457    112979.355618
1458    218885.279639
Length: 1459, dtype: float64

In [52]:
print("Coefficients:")
print(split.coef_)
print("Intercept:")
print(split.intercept_)



Coefficients:
0        -54.455161
1         92.998541
2          0.400272
3       5373.704056
4       6462.413031
           ...     
269    -2712.954020
270   -17649.811913
271   -12889.083963
272    -8094.816934
273    17660.104342
Length: 274, dtype: float64
Intercept:
-304731.2253838906


In [53]:
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(Y.astype('int64'),y_pred_split.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(Y.astype('float32'),y_pred_split.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(Y.astype('int64'),y_pred_split.astype('int64')))

MSE:
12504370462.423578
R2 Score:
-0.9813649654388428
MAE:
82249.70938999315


In [54]:
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']
for i in algorithm:
    print("Algorithm:")
    print(i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    reg = lr.fit(X,Y)
    preds = lr.predict(Test)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(Y.astype('int64'),y_pred_split.astype('int64')))
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(Y.astype('float32'),y_pred_split.astype('float32')))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(Y.astype('int64'),y_pred_split.astype('int64')))

Algorithm:
svd
MSE:
12504370462.423578
R2 Score:
-0.9813649654388428
MAE:
82249.70938999315
Algorithm:
eig
MSE:
12504370462.423578
R2 Score:
-0.9813649654388428
MAE:
82249.70938999315
Algorithm:
qr
MSE:
12504370462.423578
R2 Score:
-0.9813649654388428
MAE:
82249.70938999315
Algorithm:
svd-qr
MSE:
12504370462.423578
R2 Score:
-0.9813649654388428
MAE:
82249.70938999315
Algorithm:
svd-jacobi
MSE:
12504370462.423578
R2 Score:
-0.9813649654388428
MAE:
82249.70938999315


without train test split 

In [55]:
lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = 'svd-jacobi')
reg = lr.fit(X.values,Y)
y_pred=lr.predict(Test)
y_pred

array([113100.71564199, 161440.16344747, 189511.19467818, ...,
       180949.85461965, 104642.18062259, 220676.11610316])

In [56]:
print("R-Squared Value for Training Set:",lr.score(X.astype('float32'), Y.astype('float32')))

R-Squared Value for Training Set: 0.930626630783081


In [57]:
print("Coefficients:")
print(reg.coef_)
print("Intercept:")
print(reg.intercept_)



Coefficients:
[-4.20362982e+01  5.40500448e+01  7.42416875e-01  6.37803480e+03
  5.67609373e+03  3.18199204e+02  9.92670529e+01  2.03619771e+01
 -7.77412548e+03 -7.78452528e+03 -7.79361340e+03  7.80852900e+03
  1.38819514e+04  1.38971392e+04  1.38457020e+04 -1.38306540e+04
  6.47273946e+02 -6.09753889e+02  4.01278958e+03  1.07075926e+03
 -3.41765335e+03 -1.23358572e+04  9.65101793e+02  2.67680612e+03
  2.81084246e+03  1.50534732e+01  1.36567652e+01  5.79852735e+00
  5.17312193e+00  3.18663645e+01  2.89520109e+01  8.40668888e+01
  1.00025737e-01 -4.40621221e+02 -4.19593087e+02 -1.27831703e+04
  2.17713318e+04  1.37328767e+04  1.54045600e+04  1.12957486e+04
 -2.92162966e+04  1.49582784e+03 -4.28123135e+03 -6.48097659e+02
 -3.93425283e+01 -2.48139096e+03 -8.40285897e+03  3.31688350e+02
 -1.73893666e+04 -2.09072372e+03  3.74649602e+02 -3.09214816e+04
  1.42141834e+04  2.29372945e+04  6.50128893e+03 -6.24765239e+01
  1.29633554e+04  2.62725690e+04  3.27123268e+04 -1.76289052e+04
  8.0738857

In [58]:
print("MSE:")
print(cuml.metrics.regression.mean_squared_error(Y.astype('int64'),y_pred_split.astype('int64')))
print("R2 Score:")
print(cuml.metrics.regression.r2_score(Y.astype('float32'),y_pred_split.astype('float32')))
print("MAE:")
print(cuml.metrics.regression.mean_absolute_error(Y.astype('int64'),y_pred_split.astype('int64')))

MSE:
12504370462.423578
R2 Score:
-0.9813649654388428
MAE:
82249.70938999315


In [59]:
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']
for i in algorithm:
    print("Algorithm:")
    print(i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    reg = lr.fit(X,Y)
    preds = lr.predict(Test)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(Y.astype('int64'),y_pred.astype('int64')))
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(Y.astype('float32'),y_pred.astype('float32')))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(Y.astype('int64'),y_pred.astype('int64')))

Algorithm:
svd
MSE:
12518006649.93009
R2 Score:
-0.9835253953933716
MAE:
82414.24811514736
Algorithm:
eig
MSE:
12518006649.93009
R2 Score:
-0.9835253953933716
MAE:
82414.24811514736
Algorithm:
qr
MSE:
12518006649.93009
R2 Score:
-0.9835253953933716
MAE:
82414.24811514736
Algorithm:
svd-qr
MSE:
12518006649.93009
R2 Score:
-0.9835253953933716
MAE:
82414.24811514736
Algorithm:
svd-jacobi
MSE:
12518006649.93009
R2 Score:
-0.9835253953933716
MAE:
82414.24811514736


In [60]:
sample_sub = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
sample_sub.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [61]:
sample_sub['SalePrice'] = y_pred_split
sample_sub.to_csv('submissionshrey.csv', index=False)